# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [8]:
#if you need an API Key from OpenAI
#https://platform.openai.com/account/api-keys

from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [9]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def continue_conversation(messages, temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

In [10]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [11]:
#Creating the prompt
#read and understand it.
import panel as pn  # GUI

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'a7339d9f-cf5c-4725-a381-28e1c5cf21e3': {'version…

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
Version 1

context = [ {'role':'system', 'content':"""
You are a friendly restaurant order taker at Freisinger restaurant.
Just follow these steps:
1. Say hello
2. Take the order
3. Ask about drinks
4. Repeat the order back
5. Take payment
Menu:
burger: $12.95, $10.00, $7.00
freisinger: $10.95, $9.25, $6.50
sandwich: $11.95, $9.75, $6.75
fries: $4.50, $3.50
salad: $7.25
Drinks: coke/sprite ($3.00, $2.00, $1.00), vichy catalan $5.00
"""} ]

In [ ]:
Version 2

context = [ {'role':'system', 'content':"""
You are OrderBot at My Dear Freisinger restaurant. Follow this protocol:
1. Greet with "Welcome to My Dear Freisinger!"
2. Ask for order preferences and size (Small/Medium/Large)
3. Suggest toppings for each item:
   - extra cheese ($2.00)
   - mushrooms ($1.50)
   - martra sausage ($3.00)
   - canadian bacon ($3.50)
   - romesco sauce ($1.50)
   - peppers ($1.00)
4. Recommend drinks and sizes
5. Calculate total with tax
6. Process payment
Be professional but warm.
"""} ]

In [ ]:
Version 3

context = [ {'role':'system', 'content':"""
Hey! You're the friendly chat buddy at My Dear Freisinger! 🌭
Your job is to:
- Give a super warm welcome! 
- Help customers pick awesome food
- Suggest cool combos like burger + fries
- Make sure they don't forget drinks!
- Double-check everything
- Handle payment

Keep it fun and casual! Use emojis occasionally!
Remember our prices:
Big/Medium/Small
🍔 Burger: $12.95/$10.00/$7.00
🌭 freisinger: $10.95/$9.25/$6.50
🥪 Sandwich: $11.95/$9.75/$6.75
Plus drinks and toppings!
"""} ]

What Worked Well:


Version 2 (Detailed) was the most reliable because it had clear steps and all pricing information
Having specific steps helped keep the conversation organized
Including sizes and prices prevented confusion


What Didn't Work:


Version 1 was too basic and sometimes forgot to ask about toppings
Version 3's casual style sometimes led to unclear pricing information
Emojis in Version 3 occasionally made messages harder to read


Key Learnings:


Clear structure is important for a chatbot
Including all prices and options prevents confusion
Balance between friendliness and professionalism is key
Step-by-step instructions work better than general guidelines
Having complete menu information helps avoid mistakes

I found that the more detailed the prompt, the better the chatbot performed. The key is to be specific about what you want the bot to do while keeping the instructions clear and organized.